# Ритейл — Анализ программы лояльности

ОПИСАНИЕ ДАННЫХ:\
Датасет содержит данные о покупках в магазине строительных материалов «Строили, строили и наконец построили». \
Все покупатели могут получить доступ в магазин с использованием персональных карт. \
За 200 рублей в месяц они могут стать участниками программы лояльности. \
В программу включены скидки, специальные предложения, подарки.

ЗАДАЧИ ПРОЕКТА:\
Провести исследовательский анализ данных;\
Провести анализ программы лояльности;\
Сформулировать и проверить статистические гипотезы.\
Оценить возможности развития программы лояльности.

## Загрузка данных.
Загрузка и первичный осмотр, получение общей информации о наборах данных.

В нашем распоряжении два датасета:\
retail_dataset.csv - журнал продаж\
product_codes.csv - стоимость товара

retail_dataset.csv:

- `purchaseId` — id чека;
- `item_ID` — id товара;
- `purchasedate` — дата покупки;
- `Quantity` — количество товара;
- `CustomerID` — id покупателя;
- `ShopID` — id магазина;
- `loyalty_program` — участвует ли покупатель в программе лояльности;

product_codes.csv:

- `productID` — id товара;
- `price_per_one` — стоимость одной единицы товара;

In [246]:
#загрузим нужные библиотеки
import pandas as pd
import numpy as np
from numpy import mean
import math as mth
import datetime as dt
import seaborn as sns
import scipy.stats as st
import plotly.express as px
import matplotlib.pyplot as plt
from plotly import graph_objects as go

In [247]:
#загрузим данные
product, retail = (
pd.read_csv("C:\\Users\\kirio\\OneDrive\\Рабочий стол\\ЯПрактикум\\14_Выпускной Проект\\product_codes.csv", sep=','), #стоимость товара
pd.read_csv('C:\\Users\\kirio\\OneDrive\\Рабочий стол\\ЯПрактикум\\14_Выпускной Проект\\retail_dataset.csv', sep=',') #журнал продаж
)
  

In [248]:
#общая информация о наборе данных в журнале продаж
retail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105335 entries, 0 to 105334
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   purchaseid       105335 non-null  object 
 1   item_ID          105335 non-null  object 
 2   Quantity         105335 non-null  int64  
 3   purchasedate     105335 non-null  object 
 4   CustomerID       69125 non-null   float64
 5   ShopID           105335 non-null  object 
 6   loyalty_program  105335 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 5.6+ MB


В датасете журнала продаж 105335 строк, 7 столбцов. Три типа данных: object, float64 и int64. В столбце 'CustomerID' есть пропущенные значения. 

In [249]:
#общая информация о наборе данных стоимости товара
product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9969 entries, 0 to 9968
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   productID      9969 non-null   object 
 1   price_per_one  9969 non-null   float64
dtypes: float64(1), object(1)
memory usage: 155.9+ KB


В датасете стоимости товара 9969 строк, два столбца. Два типа данных: object и float64. Пропусков нет.

## Предобработка данных.
Преобразование типов данных. Переименование и добавление столбцов. Работа с пропусками и дубликатами в данных.  

Исследовать пропущенные значения;\
Исследовать соответствие типов;\
Исследовать дубликаты;\
Проверить корректность наименований колонок;\
Переименовать колонки;\
Удалить дубликаты;\
Привести типы;\
Заменить пропущенные значения.

In [250]:
#переименуем названия столбцов
retail = retail.rename(
   columns={
       'purchaseid': 'purchase_id',\
       'item_ID': 'item_id',\
       'Quantity': 'quantity',\
       'purchasedate': 'purchase_date',\
       'CustomerID': 'customer_id',\
       'ShopID': 'shop_id'\
   }
)

In [251]:
#количество пропусков в столбце customer_id
retail['customer_id'].isna().sum()

36210

In [252]:
# % пропусков
pd.DataFrame(round(retail.isna().mean()*100,)).style.background_gradient('coolwarm')

,0
purchase_id,0.000000
item_id,0.000000
quantity,0.000000
purchase_date,0.000000
customer_id,34.000000
shop_id,0.000000
loyalty_program,0.000000


In [253]:
#срез по незарегистрированным пользователям не участвующих в программе лояльности 
retail.query('loyalty_program == 0.0')['customer_id'].isna().sum()

36210

Треть (34%) пользователей совершили покупки без регистрации (количество: 36210) и все они не участвуют в программе лояльности. На данном этапе эти данные оставим.

In [254]:
#количество событий в разрезе программы лояльности
retail['loyalty_program'].value_counts()

0.0    81493
1.0    23842
Name: loyalty_program, dtype: int64

In [255]:
#выведем количество уникальных пользователей по группе лояльности
retail.groupby('loyalty_program').agg({'customer_id':'nunique'})

,customer_id
loyalty_program,
0.0,1162
1.0,587


Гипотезы основаны на следующих метриках: средний чек, среднее количество покупок и кумулятивная сумма денег которую  клиент в среднем принёс компании.\
Все они основаны на расчетах на каждого клиента, в этом случае отсутствующие значения могут быть интерпретированы как действия одного клиента с множеством покупок.\
Учитывая 4х кратное превосходство количества событий не участвующих в программе лояльности и двукратное уникальных пользователей, мы удалим строки с отсутствующими значениями.\
Несмотря на значительную потерю данных, датасет будет более релевантен нашим исследованиям.

In [256]:
retail

,purchase_id,item_id,quantity,purchase_date,customer_id,shop_id,loyalty_program
0,538280,21873,11,2016-12-10 12:50:00,18427.0,Shop 0,0.0
1,538862,22195,0,2016-12-14 14:11:00,22389.0,Shop 0,1.0
2,538855,21239,7,2016-12-14 13:50:00,22182.0,Shop 0,1.0
3,543543,22271,0,2017-02-09 15:33:00,23522.0,Shop 0,1.0
4,543812,79321,0,2017-02-13 14:40:00,23151.0,Shop 0,1.0
...,...,...,...,...,...,...,...
105330,538566,21826,1,2016-12-13 11:21:00,NaN,Shop 0,0.0
105331,540247,21742,0,2017-01-05 15:56:00,21143.0,Shop 0,0.0
105332,538068,85048,1,2016-12-09 14:05:00,23657.0,Shop 0,1.0
105333,538207,22818,11,2016-12-10 11:33:00,18427.0,Shop 0,0.0


In [257]:
#преобразуем типы данных в столбцах customer_id и purchase_date, отсутствующие зачения customer_id 
retail['customer_id'] = retail['customer_id'].fillna(0).astype('int')
#loyalty_program
retail['purchase_date'] = pd.to_datetime(retail['purchase_date']).dt.date

In [258]:
#минимальная дата привлечения
#profiles['dt'].min()


# преобразование данных о времени
#costs['dt'] = pd.to_datetime(costs['dt']).dt.date

#Год выхода игр 'year_of_release' и оценки критиков 'critic_score' переведем из вещественного типа в целочисленный
#data['year_of_release'] = data['year_of_release'].astype('int')
#data['critic_score'] = data['critic_score'].astype('int')

#и приведем к вещественному типу 'object'
#data['user_score'] = data['user_score'].astype('float')

#удалим из самого датафрейма строки с явными дубликатами, оставив только первые вхождения: будем считать, что верные идентификаторы встречаются первыми
#data = data.drop_duplicates(subset=['name','platform'], keep='first')

In [259]:
retail

,purchase_id,item_id,quantity,purchase_date,customer_id,shop_id,loyalty_program
0,538280,21873,11,2016-12-10,18427,Shop 0,0.0
1,538862,22195,0,2016-12-14,22389,Shop 0,1.0
2,538855,21239,7,2016-12-14,22182,Shop 0,1.0
3,543543,22271,0,2017-02-09,23522,Shop 0,1.0
4,543812,79321,0,2017-02-13,23151,Shop 0,1.0
...,...,...,...,...,...,...,...
105330,538566,21826,1,2016-12-13,0,Shop 0,0.0
105331,540247,21742,0,2017-01-05,21143,Shop 0,0.0
105332,538068,85048,1,2016-12-09,23657,Shop 0,1.0
105333,538207,22818,11,2016-12-10,18427,Shop 0,0.0


In [260]:
retail.query('item_id == "21873"').head()

,purchase_id,item_id,quantity,purchase_date,customer_id,shop_id,loyalty_program
0,538280,21873,11,2016-12-10,18427,Shop 0,0.0
7584,541104,21873,0,2017-01-13,0,Shop 0,0.0
8576,540418,21873,1,2017-01-07,0,Shop 0,0.0
13679,541516,21873,2,2017-01-18,0,Shop 0,0.0
15316,541566,21873,35,2017-01-19,23401,Shop 0,1.0


In [261]:
#проверим на явные дубликаты
retail.duplicated().sum()


1034

In [262]:
retail['customer_id'].unique().sum()

36675768

In [263]:
#
retail.groupby('shop_id').agg({'purchase_id':'count'}).sort_values(by='purchase_id', ascending=False).head()


,purchase_id
shop_id,
Shop 0,97393
Shop 4,1741
Shop 1,1540
Shop 6,1032
Shop 8,560


In [264]:
#
retail.groupby('shop_id').agg({'customer_id':'nunique'}).sort_values(by='customer_id', ascending=False).head()

,customer_id
shop_id,
Shop 0,1573
Shop 4,43
Shop 1,38
Shop 8,12
Shop 12,10


In [265]:
#переименуем название столбца
product = product.rename(columns={'productID': 'item_id'})

In [266]:
product

,item_id,price_per_one
0,85123A,2.55
1,71053,3.39
2,84406B,2.75
3,84029G,3.39
4,84029E,3.39
...,...,...
9964,84306,8.29
9965,90001A,4.15
9966,90118,2.46
9967,DOT,172.62


In [267]:
product.query('item_id == "21873"')

,item_id,price_per_one
762,21873,1.25
3488,21873,3.36
4899,21873,0.42
5909,21873,1.66
7200,21873,1.63
8936,21873,3.29
9779,21873,1.06


### Исследуем пропущенные значения

# Исследовательский анализ данных